In [1]:
import os
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

In [3]:
import numpy as np
np.random.seed(2016)

In [4]:
top_model_weights_path = 'localization/localizer/inception/localizer_bottleneck_fc_model.h5'
# dimensions of our images.0
img_width, img_height = 256, 256
nb_epoch = 50

In [14]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False,input_shape = (256,256,3))


In [15]:
base_model.output

<tf.Tensor 'concat_v2_14:0' shape=(?, 6, 6, 2048) dtype=float32>

In [7]:
def load_from_file(filename):
	import numpy as np
	return np.load( filename + '.npy')

In [58]:
def get_train_val():
    coordinates = load_from_file('localization/localizer/train/coordinates_float_256')
    resize_img = load_from_file('localization/localizer/train/resize_img_float_256')
    Y = coordinates.reshape((-1,4))
    X = resize_img.reshape((-1,256,256,3))
    # 这里的数据，x是0-255之间的值，y是0-100之间的值。都是浮点数类型。
    Y = Y/256.0
    X = X/255.0
    return X, Y

In [17]:
def shuffle(data, labels):
    rnd = np.random.get_state()
    np.random.shuffle(data)
    np.random.set_state(rnd)
    np.random.shuffle(labels)
    return data, labels

In [18]:
def get_shuffle_data():
    X, Y = get_train_val()
    X, Y = shuffle(X, Y)
    return X, Y

In [61]:
X, Y = get_shuffle_data()

In [62]:
X.shape

(6598, 256, 256, 3)

In [63]:
import helpers
helpers.save_to_file('localization/localizer/train/shuffled_coordinates_float_256',Y)
helpers.save_to_file('localization/localizer/train/shuffled_resize_img_float_256',X)

In [8]:
def get_shuffled():
    coordinates = load_from_file('localization/localizer/train/shuffled_coordinates_float_256')
    resize_img = load_from_file('localization/localizer/train/shuffled_resize_img_float_256')
    Y = coordinates.reshape((-1,4))
    X = resize_img.reshape((-1,256,256,3))
    return X,Y

In [9]:
X, Y = get_shuffled()

# Step1
Fine Tune第一步是使用已经训练好的inception_v3模型提取features.即将inception_v3最后一个convBlock的输出作为features.

In [64]:
# train:
bottleneck_features_train = base_model.predict(X[660:], batch_size=32, verbose=0)
np.save(open('localization/localizer/inception/bottleneck_features_train.npy', 'w'), bottleneck_features_train)
# validation:
bottleneck_features_val = base_model.predict(X[:660], batch_size=32, verbose=0)
np.save(open('localization/localizer/inception/bottleneck_features_val.npy', 'w'), bottleneck_features_val)
    

In [11]:
from keras import backend as K
def l2_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis = -1))   

# Step 2
Fine Tune第二步是使用第一步提取的特征训练一个自定义的适用于自己要解决的问题的head. 这里是训练regression问题，所以是regression head。 将训练好的weights保存作为正式进行训练时的初始值。

In [5]:
def train_top_model():
    from keras.optimizers import SGD
    
    train_data = np.load(open('localization/localizer/inception/bottleneck_features_train.npy'))
    train_labels = Y[660:]

    validation_data = np.load(open('localization/localizer/inception/bottleneck_features_val.npy'))
    validation_labels = Y[:660]

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(BatchNormalization(axis=1))
    model.add(Dense(512, activation='relu', init='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu', init='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(4, init='he_uniform'))
    callbacks = [
            EarlyStopping(monitor='val_loss', patience=3, verbose=0)
        ]
    
    sgd = SGD(lr=1e-2, decay=1e-4, momentum=0.9, nesterov=False)

    model.compile(loss=l2_loss, optimizer=sgd)
    
    model.fit(train_data, train_labels,
              nb_epoch=20, batch_size=32, shuffle=True, verbose=2,
              validation_data=(validation_data, validation_labels), callbacks = callbacks)
    model.save_weights(top_model_weights_path)

In [12]:
train_top_model()

Train on 5938 samples, validate on 660 samples
Epoch 1/20
129s - loss: 1.0488 - val_loss: 0.5747
Epoch 2/20
114s - loss: 0.6865 - val_loss: 0.4446
Epoch 3/20
121s - loss: 0.6015 - val_loss: 0.4270
Epoch 4/20
119s - loss: 0.5642 - val_loss: 0.3486
Epoch 5/20
128s - loss: 0.4966 - val_loss: 0.3012
Epoch 6/20
121s - loss: 0.4647 - val_loss: 0.3247
Epoch 7/20
123s - loss: 0.4244 - val_loss: 0.2320
Epoch 8/20
133s - loss: 0.3884 - val_loss: 0.2600
Epoch 9/20
127s - loss: 0.3794 - val_loss: 0.2875
Epoch 10/20
125s - loss: 0.3425 - val_loss: 0.2661
Epoch 11/20
147s - loss: 0.3335 - val_loss: 0.2313
Epoch 12/20
131s - loss: 0.3086 - val_loss: 0.2388
Epoch 13/20
129s - loss: 0.2877 - val_loss: 0.1967
Epoch 14/20
130s - loss: 0.2755 - val_loss: 0.2001
Epoch 15/20
122s - loss: 0.2629 - val_loss: 0.1701
Epoch 16/20
119s - loss: 0.2526 - val_loss: 0.2240
Epoch 17/20
125s - loss: 0.2426 - val_loss: 0.1907
Epoch 18/20
115s - loss: 0.2275 - val_loss: 0.1809
Epoch 19/20
114s - loss: 0.2201 - val_loss: 

# Step 3
Fine Tune第三步是利用第二步训练好的头部weights和inception_v3模型本身的weights在自己问题上的数据集进行训练.

In [29]:
#前两步是为了节省头部的训练，但是这里目前没有找到两种结合的办法，所以暂时从头训练头部。

In [25]:
x = base_model.output
x = AveragePooling2D()(x)
x = BatchNormalization(axis=1)(x)
# let's add a fully-connected layer
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(4)(x)
# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False


In [27]:
sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=False)

model.compile(loss=l2_loss, optimizer=sgd)

train_data = X[660:]
train_labels = Y[660:]
validation_data = X[:660]
validation_labels = Y[:660]

In [28]:
ckpt = ModelCheckpoint('localization/localizer/inception/loss-{val_loss:.2f}_inception_localizer.h5', monitor='val_loss',
                       verbose=0, save_best_only=True, save_weights_only=True)
callbacks = [
            EarlyStopping(monitor='val_loss', patience=5), ckpt
        ]
model.fit(train_data, train_labels,
              nb_epoch=nb_epoch, batch_size=32, shuffle=True,
              validation_data=(validation_data, validation_labels), callbacks=callbacks)


Train on 5938 samples, validate on 660 samples
Epoch 1/50


KeyboardInterrupt: 